# Notebook 3 — Token/Context Maliyeti + CoT (Kalite vs Maliyet)

Bu notebook iki şeyi gösterir:
1) Zero/One/Few-shot prompt'larının token tüketimi
2) CoT (step-by-step) istemenin maliyet/çıktı etkisi

⏱️ Önerilen süre: **12–15 dk**

## Setup

Bu notebook'lar, LLM çağrısı için **LangChain** kullanır ve otomatik sağlayıcı seçer:

- `OPENAI_API_KEY` varsa OpenAI üzerinden gider (varsayılan model: `gpt-4o-mini`)
- `GOOGLE_API_KEY` varsa Gemini üzerinden gider (varsayılan model: `gemini-1.5-pro`)

> Dilerseniz model adını `OPENAI_MODEL` / `GEMINI_MODEL` ile değiştirebilirsiniz.

**Not:** Bu notebook dosyaları örnek amaçlıdır. Kendi kurumsal ortamınızda anahtar yönetimini güvenli şekilde yapın.

In [ ]:
# Lokal/Colab için (gerekliyse) kurulum:
%pip -q install -U langchain-core langchain-openai langchain-google-genai pydantic tiktoken python-dotenv pandas matplotlib

In [ ]:
import os, json, re
from typing import Dict, Any

def get_llm():
    # Sağlayıcı seçimi: OpenAI -> Gemini
    openai_key = os.getenv('OPENAI_API_KEY')
    google_key = os.getenv('GOOGLE_API_KEY')

    if openai_key:
        from langchain_openai import ChatOpenAI
        model = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
        llm = ChatOpenAI(model=model, temperature=0.1)
        return llm, f'OpenAI ({model})'

    if google_key:
        from langchain_google_genai import ChatGoogleGenerativeAI
        model = os.getenv('GEMINI_MODEL', 'gemini-1.5-pro')
        llm = ChatGoogleGenerativeAI(model=model, temperature=0.1, api_key=google_key)
        return llm, f'Google Gemini ({model})'

    raise RuntimeError(
        'API key bulunamadı. Lütfen ortam değişkeni tanımlayın:\n'
        '- OpenAI için: OPENAI_API_KEY\n'
        '- Gemini için: GOOGLE_API_KEY\n'
        'Opsiyonel: OPENAI_MODEL / GEMINI_MODEL'
    )

llm, provider = get_llm()
print('✅ LLM hazır:', provider)

def llm_text(prompt: str) -> str:
    resp = llm.invoke(prompt)
    return getattr(resp, 'content', str(resp)).strip()

def strip_fences(s: str) -> str:
    s = s.strip()
    s = re.sub(r'^```json\s*', '', s, flags=re.IGNORECASE)
    s = re.sub(r'^```\s*', '', s)
    s = re.sub(r'\s*```$', '', s)
    return s.strip()

In [ ]:
# Mini veri seti (email/ticket) — demo için
EMAILS = [
    {'id': 'E1', 'text': 'Kargom hâlâ gelmedi. 7 gündür bekliyorum. Acil çözüm istiyorum!', 'notes': 'Gecikme + yüksek aciliyet'},
    {'id': 'E2', 'text': 'Ürün kırık geldi. Değişim yapabilir miyiz?', 'notes': 'Hasarlı ürün'},
    {'id': 'E3', 'text': 'İade sürecini nasıl başlatabilirim? Kutuyu attım ama ürün duruyor.', 'notes': 'İade + edge-case (kutusuz)'},
    {'id': 'E4', 'text': 'Kartımdan iki kez çekim yapılmış görünüyor. Lütfen hemen kontrol edin.', 'notes': 'Faturalama + yüksek aciliyet'},
    {'id': 'E5', 'text': 'Ürününüzün kullanım kılavuzunu paylaşır mısınız?', 'notes': 'Bilgi talebi (low)'},
]
len(EMAILS)

## 1) Token sayımı

OpenAI için `tiktoken` daha doğru sayım verir. Diğer sağlayıcılarda tokenizer farklı olabilir.
Burada:
- `tiktoken` çalışırsa onu kullanır
- çalışmazsa yaklaşık tahmin yapar (1 token ≈ 4 karakter)

In [ ]:
import os

def count_tokens(text: str, model_hint: str = 'gpt-4o-mini') -> int:
    try:
        import tiktoken
        enc = tiktoken.encoding_for_model(model_hint)
        return len(enc.encode(text))
    except Exception:
        return max(1, len(text) // 4)

def estimate_cost(tokens_in: int, tokens_out: int, price_in_per_1k: float, price_out_per_1k: float) -> float:
    return (tokens_in/1000)*price_in_per_1k + (tokens_out/1000)*price_out_per_1k

## 2) Üç stratejinin prompt token farkı

Notebook 1'deki prompt'ların kısa bir kopyası.

In [ ]:
import json

SCHEMA = (
    'Return ONLY valid JSON with exactly these keys:\n'
    '{\n'
    '  "category": "string",\n'
    '  "urgency": "low|medium|high",\n'
    '  "reason": "string (max 1 sentence)"\n'
    '}\n'
    'No extra text, no markdown, JSON only.'
)

ONE_EXAMPLE = {
    'email': 'My order arrived broken. I want a replacement as soon as possible.',
    'answer': {'category':'Damaged product','urgency':'medium','reason':'Customer reports a damaged item and requests a replacement.'}
}

FEW_EXAMPLES = [
    ('Where is my package? It was supposed to arrive 5 days ago.', {'category':'Delivery issue','urgency':'high','reason':'Customer reports a significantly delayed delivery.'}),
    ('How can I return the product? I changed my mind.', {'category':'Return request','urgency':'low','reason':'Customer asks for return instructions without a critical issue.'}),
    ('You charged me twice for the same order. Fix this immediately.', {'category':'Billing issue','urgency':'high','reason':'Customer reports a double charge and requests urgent resolution.'}),
]

def prompt_zero(email_text: str) -> str:
    return ('You are a customer support triage assistant.\n'
            'Classify the email into a category and urgency.\n\n' + SCHEMA + '\n\n' + 'Email:\n' + email_text)

def prompt_one(email_text: str) -> str:
    return ('You are a customer support triage assistant.\n'
            'Use the example to follow the same output format and labeling style.\n\n'
            + SCHEMA + '\n\n'
            + 'Example:\nEmail:\n' + ONE_EXAMPLE['email'] + '\n'
            + 'Answer:\n' + json.dumps(ONE_EXAMPLE['answer'], ensure_ascii=False) + '\n\n'
            + 'Now classify this email:\n\nEmail:\n' + email_text)

def prompt_few(email_text: str) -> str:
    ex_block = ''
    for mail, ans in FEW_EXAMPLES:
        ex_block += 'Email:\n' + mail + '\nAnswer:\n' + json.dumps(ans, ensure_ascii=False) + '\n\n'
    return ('You are a customer support triage assistant.\n'
            'Follow the same pattern as the examples.\n\n' + SCHEMA + '\n\n'
            + 'Examples:\n' + ex_block
            + 'Now classify this email:\n\nEmail:\n' + email_text)

### Token ölçümü

In [ ]:
import pandas as pd

sample = EMAILS[0]['text']
model_hint = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')

p0, p1, p2 = prompt_zero(sample), prompt_one(sample), prompt_few(sample)
t0, t1, t2 = count_tokens(p0, model_hint), count_tokens(p1, model_hint), count_tokens(p2, model_hint)

pd.DataFrame([
    {'strategy':'zero-shot', 'prompt_tokens': t0},
    {'strategy':'one-shot',  'prompt_tokens': t1},
    {'strategy':'few-shot',  'prompt_tokens': t2},
]).sort_values('prompt_tokens')

## 3) Basit maliyet simülasyonu

Fiyatlar modele göre değişir. Buraya kendi fiyatlarını gir.
Çıktı token'ı JSON kısa olduğundan genelde 60–120 arası olur (tahmin).

In [ ]:
price_in_per_1k  = 0.0   # örn: 0.15
price_out_per_1k = 0.0   # örn: 0.60
avg_out_tokens   = 90

rows = []
for name, tok in [('zero-shot', t0), ('one-shot', t1), ('few-shot', t2)]:
    rows.append({
        'strategy': name,
        'prompt_tokens_in': tok,
        'avg_tokens_out': avg_out_tokens,
        'est_cost_per_call': estimate_cost(tok, avg_out_tokens, price_in_per_1k, price_out_per_1k)
    })
pd.DataFrame(rows)

## 4) Context şişmesi (örnek sayısı arttıkça)

Few-shot örnek sayısını artırınca prompt token'ı nasıl büyüyor?

In [ ]:
def prompt_with_n_examples(email_text: str, n: int) -> str:
    exs = (FEW_EXAMPLES * ((n // len(FEW_EXAMPLES)) + 1))[:n]
    ex_block = ''
    for mail, ans in exs:
        ex_block += 'Email:\n' + mail + '\nAnswer:\n' + json.dumps(ans, ensure_ascii=False) + '\n\n'
    return ('You are a customer support triage assistant.\n'
            'Follow the same pattern as the examples.\n\n' + SCHEMA + '\n\n'
            + 'Examples:\n' + ex_block
            + 'Now classify this email:\n\nEmail:\n' + email_text)

for n in [0, 1, 3, 5, 10, 20, 40]:
    p = prompt_with_n_examples(sample, n)
    print(f'n={n:>2}  prompt_tokens≈{count_tokens(p, model_hint)}')

## 5) CoT (Chain-of-Thought): kalite vs maliyet

Burada iki yaklaşımı karşılaştırıyoruz:
- **Direct**: sadece sonucu üret
- **Step-by-step**: adım adım çöz

> Üretimde full CoT’yi kullanıcıya göstermek istemeyebilirsiniz. Alternatif: kısa gerekçe + self-check.

In [ ]:
MATH_TASK = ('A company sold 120 items on Monday, 95 on Tuesday, and 110 on Wednesday. '
             'If 15 items were returned, how many net items were sold?')

PROMPT_DIRECT = 'Answer with a single line: "Net sold = <number>". Question: ' + MATH_TASK
PROMPT_COT    = 'Solve step by step, then answer with a single line: "Net sold = <number>". Question: ' + MATH_TASK

direct = llm_text(PROMPT_DIRECT)
cot    = llm_text(PROMPT_COT)

print('DIRECT:\n', direct)
print('\nCOT:\n', cot)

print('\nToken estimates (prompt only):')
print('direct:', count_tokens(PROMPT_DIRECT, model_hint))
print('cot   :', count_tokens(PROMPT_COT, model_hint))

## 6) Alternatif: brief rationale + self-check

CoT yerine, üretimde daha güvenli ve kısa bir format:

In [ ]:
PROMPT_BRIEF = (
    'Return:\n'
    '1) Answer: Net sold = <number>\n'
    '2) Rationale: (max 2 sentences)\n'
    '3) Self-check: (max 2 bullets)\n\n'
    'Question: ' + MATH_TASK
)

brief = llm_text(PROMPT_BRIEF)
print(brief)
print('\nPrompt token estimate:', count_tokens(PROMPT_BRIEF, model_hint))

## 7) Egzersiz (3–5 dk)

1) CoT prompt’unu Türkçe yapıp token sayımını kıyaslayın.
2) 10–20 örneğe çıkınca context’in nasıl şiştiğini not edin.
3) Brief + self-check formatını farklı bir problemde deneyin.